In [1]:
## Access and retrieve Company Level Financial data from SEC EDGAR system
##   M Boldin March-April 2023 
##   UPWORK job  for Rueben Reuben Vandeventer, Second Sight Data Science

## Produces CSV file 
##   filename == {symbol} + {”_”} + {reportingYear} + {”_”} + {analysisDate} + {”_FY}

In [1]:
## IMPORT Python modules 
##  M Boldin APRIL-MAY 2023

import os
import sys
sys.path.append("/path/to/secpy")
import datetime as dt
import numpy as np
import pandas as pd

print('Operating System:', sys.platform, '\n Python version:', sys.version)
print('** Main MODULE IMPORTs Done **', dt.datetime.now())

Operating System: darwin 
 Python version: 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
** Main MODULE IMPORTs Done ** 2023-07-05 13:13:07.823935


In [18]:
##  My extra code

import secpy as sec
from secpy.secpy_client import SECPyClient

from SecUtils import *
##  SecUtils.py can be in different directory 
##   use os.sys.path.append("./Util") to add location to Python system path 

print('** Extra  MODULE IMPORTs Done **', dt.datetime.now()) 

** Extra  MODULE IMPORTs Done ** 2023-06-18 19:22:13.158932


In [19]:
## Jupyter specific set up

pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)    

## Interactive Shells alows a cell to show more than one output cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
## Need a SEC Edgar API user code

client = SECPyClient("MDBtest")
print(client.user_agent)



MDBtest


In [21]:
## Master function to call download steps and process data 
##  Uses SECitems.csv for defining variable terms and renaming

def RunSECsteps(tsymbol=None,last=1,form='10-K',items='SECitems.csv', make_csv=True):

    ## Use CSV file to define items
    dfSECitems = pd.read_csv('SECitems.csv')
    ## Renaming dictionary used below
    c1 = dfSECitems['ConceptNameFY'].to_list()
    c2 = dfSECitems['ItemFY'].to_list()
    drename = dict(zip(c1,c2))
 
    ## Identify Concept items and Calc items
    dfx = dfSECitems.copy()
    rx = dfx['type'] == 'calc'
    dfx2 = dfx[rx]
    concepts1 = dfx.loc[~rx,'ConceptNameFY']
    concepts1 = concepts1.to_list()
 

    ## Loop -- Get all needed Company Concept Items 
    company_facts = client.company_facts()
    company1 = company_facts.get_company_facts_for_ticker(tsymbol)
    #print(tsymbol, company1.cik, company1.entity_name)
    clist = concepts1[:]
    dfB1 = ConceptLoop2(tsymbol,company1,clist,form='10-K',last=last)

    ## Restate to item names
    dfB2 = RestateSEC(tsymbol,dfB1)

    ## PIVOT  make concept-name individual column names
    dfa = dfB2.copy()
    jx = ['analysisDate', 'symbol', 'reportingDate', 'reportingPeriod', 'reportingYear', 'endDate']
    dfa['case'] = dfa['concept_name']
    dfb = dfa.pivot_table(values = 'value', index=jx, columns = 'case')
    dfb.reset_index(inplace=True)
    dfb.columns = dfb.columns.to_flat_index()
    dfB3 = dfb.copy()

    ## List of extracted items used to see what is missing 
    ca = dfB3.columns
    #print(ca)
    #print(dfB3.tail(2).T)

    ## Make Depreciation CapEx and FCF and AccumDeprec
    dfB3['AccumDepreciation'] = 0
    dfB3['Amortization'] = 0
    dfB3['CapitalExpenditures']  = 0
    dfB3['FreeCashFlow']  = 0


    ##Net PPE lag and change
    if 'PropertyPlantAndEquipmentNet' in ca:
        dfB3['NetPPE_lag'] = dfB3['PropertyPlantAndEquipmentNet'].shift(1)
        dfB3['NetPPE_chg'] = dfB3['PropertyPlantAndEquipmentNet'] - dfB3['NetPPE_lag']
    if 'PropertyPlantAndEquipmentAdditions' in ca:
        dfB3['CapitalExpenditures'] = dfB3['PropertyPlantAndEquipmentAdditions']
    else:
        net = 0; dep = 0; psale=0;
        if 'NetPPE_chg' in dfB3.columns:
            net = dfB3['NetPPE_chg']
        if 'Depreciation' in dfB3.columns:
            dep = dfB3['Depreciation']
        elif 'DepreciationAndAmortization' in dfB3.columns:
            dep = dfB3['DepreciationAndAmortization']
        if 'ProceedsFromSaleOfPropertyPlantAndEquipment' in dfB3.columns:
            psale = dfB3['ProceedsFromSaleOfPropertyPlantAndEquipment'] 
        dfB3['CapitalExpenditures'] = net  + dep + psale

    if 'NetCashProvidedByUsedInOperatingActivities'	 in ca:
        dfB3['FreeCashFlow'] = dfB3['NetCashProvidedByUsedInOperatingActivities'] -  dfB3['CapitalExpenditures'] 
    #if 'NetCashProvidedByUsedInOperatingActivities'	 in ca:

    if 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment' in ca:
        dfB3['AccumDepreciation'] = dfB3['AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment']
    elif ('PropertyPlantAndEquipmentNet' in ca) and ('PropertyPlantAndEquipmentGross' in ca):
        dfB3['AccumDepreciation'] = dfB3['PropertyPlantAndEquipmentGross'] - dfB3['PropertyPlantAndEquipmentNet'];

    if 'AmortizationOfIntangibleAssets' in ca:
        dfB3['Amortization'] = dfB3['AmortizationOfIntangibleAssets']
    elif ('Depreciation' in dfB3.columns) and ('DepreciationAndAmortization' in dfB3.columns):
        dfB3['Amortization'] = dfB3['DepreciationAndAmortization'] - dfB3['Depreciation']

    if 'Depreciation' not in ca:
        dfB3['Depreciation'] = 0
        
    ## Rename SEC concept_name to Item FY tag 
    ##   rename dictionary defined above
    c1 = dfB3.columns
    c2 = list(c1)
    for k,c in enumerate(c1):
        #print(k, c)    
        if (c in drename.keys()) and  (drename[c] != 'extra'): 
            #print(k, c, drename[c])
            c2[k] = drename[c]
    dfB4 = dfB3.copy()
    dfB4[c2] = dfB4[c1]

    ## Re-oder and finish
    
    ca = ['analysisDate', 'symbol', 'reportingDate', 'reportingPeriod','reportingYear', 'endDate']
    
    cb = ['earningsFY',
    'cummDepreciationFY',
    'depreciationExpenseFY',
    'amortizationFY',
    'totalCapexFY',
    'operatingCashFlowFY',
    'freeCashFlowFY']
    if 'sharesOutstandingFY' in dfB4.columns: 
        cb.append('sharesOutstandingFY')

    k = dfB4.shape[0]
    if last==1:
        dfB5 = dfB4.loc[k-1:,ca+cb]
    else:
        dfB5 = dfB4.loc[k-last:k,ca+cb]    
        
    if make_csv == True:
        x=dfB5.iloc[-1,:]
        symbol = x['symbol']
        reportingYear = x['reportingYear']
        analysisDate = x['analysisDate']
        filename = f'{symbol}_{reportingYear}_{analysisDate}_FY.csv'
        print('Creating:', filename)
        dfB5.to_csv(filename)
        
    return dfB5

In [25]:
## Example extracts

MSFT = RunSECsteps('MSFT',last=2, form='10-K')
MSFT.T

Creating: MSFT_2022_2023-06-18_FY.csv


,4,5
case,,
analysisDate,2023-06-18,2023-06-18
symbol,MSFT,MSFT
reportingDate,2021-07-29,2022-07-28
reportingPeriod,FY,FY
reportingYear,2021,2022
endDate,2021-06-30,2022-06-30
earningsFY,61271000000.0,72738000000.0
cummDepreciationFY,51351000000.0,59660000000.0
depreciationExpenseFY,9300000000.0,12600000000.0
